In [1]:
import networkx as nx
import numpy as np
from scipy.io import loadmat
import os

In [2]:
links = [[0,1],[1,2],[2,3],[2,4],[3,5],[5,6],[5,7],[0,8],[8,9],[8,10],[10,11],[11,12],[12,13],[12,14],[14,15]] 
OUTPUT_FILE = "results_tree_network_khop5_LBA.csv"
k = 5
NUM_TREES_PER_FILE = 1
NUM_SOURCES_LIST =[10]

In [3]:
def nodes_at_distance(G_down, root):
    nodes_dict = {}
    visited = set()

    def dfs(node, distance):
        visited.add(node) 
        nodes_dict[distance] = nodes_dict.get(distance, 0) + 1

        for neighbor in G_down.neighbors(node):
            if neighbor not in visited:
                dfs(neighbor, distance + 1)
    dfs(root, 1)
    return nodes_dict

In [4]:
outfile = open(OUTPUT_FILE, "w")

for num_sources in NUM_SOURCES_LIST:
    for tree_index in range(NUM_TREES_PER_FILE):
        print("%u, %u\n" % (num_sources, tree_index))
        
        G_up = nx.DiGraph()
        G_down = nx.DiGraph()
        root = 0
        for l in links:
            G_up.add_edge(l[1], l[0])
            G_down.add_edge(l[0], l[1])
        subtree_roots = [n for n in nx.neighbors(G_down, 0)]
        
        leaf_nodes = []
        for n, d in G_up.in_degree:
            if d == 0:
                leaf_nodes.append(n)

        line_graphs = []
        for n in sorted(leaf_nodes):
            line_graphs.append(nx.shortest_path(G_up, n, 0))

        hop_distance = 0
        for n in G_up.nodes:
            if n == 0:
                continue
            hop_distance = hop_distance + nx.shortest_path_length(G_up, n, 0)
        avg_hop_distance = hop_distance / (len(G_up.nodes) - 1)
        
        nb = []
        for n in subtree_roots:
            nb.append(len(nx.descendants(G_down, n)) + 1)

        ni_ = [[0] for _ in range(len(subtree_roots))]
        max_hop_is = []
        for m, u in enumerate(subtree_roots):
            nodes_count = nodes_at_distance(G_down, u)
            values_list = list(nodes_count.keys())
            max_hop_is.append(np.max(values_list))

            sum_is = 0  
            for j in range(1, min([int(np.ceil(k / 2)), max_hop_is[m]]) + 1):
                if j in nodes_count:
                    sum_is += nodes_count[j]
                else:
                    nodes_count[j] = 0
                ni_[m].append(sum_is)


        eb = []
        for i in range(len(nb)):
            ebt = 0
            for j in range(1, min([int(np.ceil(k/2)), max_hop_is[i]]) + 1):
                ebt = ebt + (nb[i] - ni_[i][j - 1])
            eb.append(ebt)


        lg_length = []
        max_length_subtree_indx = [] 

        for root in subtree_roots:
            max_length = -1
            max_length_subtree = None
            for w,lg in enumerate(line_graphs):
                if root in lg:
                    length = len(lg)
                    if length > max_length:
                        max_length = length
                        max_length_subtree = w
            max_length_subtree_indx.append(max_length_subtree)
            lg_length.append(max_length)


        fb = []
        for w, m in enumerate(subtree_roots):
            ind = max_length_subtree_indx[w]
            line_is = line_graphs[ind]
            fbt = 0
            for j in range(int(np.ceil(k / 2)) + 1, min(k+1, lg_length[w]-1)+1):
                nodeis = line_is[-1-j]
                descendants = len(nx.descendants(G_down, nodeis)) + 1
                fbt += descendants
            fb.append(fbt)

        
        
        eb_matrix = np.tile(eb, [len(fb), 1])
        eb_matrix[np.where(np.eye(len(fb)))] = 0
        eb_other = np.sum(eb_matrix, axis = 1)
        
        extra_slots = []
        for i in range(len(fb)):
            extra_slots.append(max(fb[i] - eb_other[i], 0))
            
        extra = max(extra_slots)
        print('extra',extra)
        lb = int(np.sum(eb) + extra)
        print("framelb = ", lb)
        
        aaoi_min = avg_hop_distance + (lb - 1)/2
        print('Nframe', lb,'aaoi_min',aaoi_min)
        outfile.write("%u, %u, %u, %f\n" % (num_sources, tree_index, lb, aaoi_min))

outfile.close()

10, 0

extra 0
framelb =  38
Nframe 38 aaoi_min 21.9
